# Analysis of in-text-citations

In this project I explore the different citation patterns of successful and unssesful papers. I use a dataset that includes information on 156K papers from the PLoS journals (Public Library of Science) between 2005 and 2016, citing 2.5M unique papers, and making up to 6.8M records in total (each record is a pair paper-reference).

First, I read the datafile and I do some exploratory statistical analysis. 

Next, I plot several figures, for example, the age of the references used by a paper, as a function of the paper section as well as the future impact (number of citations) of the paper itself. I control for publication year and subject field. Similarly, I obtain the corresponding plot for the number of citations of the references used by PLoS papers.

Finally, I run a null-model to compare the actual usage of top and botoom references by top and bottom PLoS papers ('top' and 'bottom' defined by the percentile of number of citations received in its cohort year), to the expected values. The randomization process preserves certain basic properties of the data, such as publication year, subject field, and groups of references cited together in the same cluster in a given paper. I then plot the actual vs expected values, as well as computing the corresponding z-scores.


In [1]:
import sys

import matplotlib.pyplot as plt
%matplotlib inline
import copy
import pickle
import gzip
import os,glob
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats
import math
import time
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff
import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



####### to make the notebook use the entire width of the browser
from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  

## Uploading files

In [ ]:
#########  upload the main datafile        

%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)



######  upload auxiliary file with info only on plos paper 

%time plos_df = pickle.load(open('../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect_young_old.pkl', 'rb'))
print ("done loading plos_df", plos_df.shape)





######## i load the dictionary for category-code

dict_categ_code = pickle.load(open('../data/dict_categ_code.pkl', 'rb'))


print ("from the entire df ",df_merged.shape, "; and plos one records:",  df_merged[df_merged['plos_j1']== "PLOS ONE"].shape,"\n\n")
# {'Biology and life sciences': 0,
#  'Computer and information sciences': 1,
#  'Earth sciences': 2,
#  'Ecology and environmental sciences': 3,
#  'Engineering and technology': 4,
#  'Medicine and health sciences': 5,
#  'People and places': 6,
#  'Physical sciences': 7,
#  'Research and analysis methods': 8,
#  'Science policy': 9,
#  'Social sciences': 10}

dict_code_categ={}
dict_size_categ={}
for categ in dict_categ_code:
    code = str(dict_categ_code[categ])
    
    df_selection_categ = df_merged[df_merged['categ_codes'].str.contains(code)]
   # print (categ, code, df_selection_categ.shape)
    size= len(df_selection_categ)
    dict_size_categ[size] = categ
    dict_code_categ[code] = categ

for size in reversed(sorted(dict_size_categ)):
    print (size, dict_size_categ[size])

    
    

df_merged.head()

## Some basic data exploration


In [ ]:
###### basic data exploration:
##########################################

#ALL sections:  {'intro': 0, 'methods': 1, 'results': 2, 'disc': 3, 'res_disc':4, 'concl':5, 'mixed':6, 'na':7}

# df_merged.regex_sect_index.value_counts()
# 0    2188159 / 5787630. =  0.37807
# 3    2105019 / 5787630. =  0.36371
# 1     678770 / 5787630. =  0.11728
# 2     563650 / 5787630. =  0.09739
# 4     199399 / 5787630. =  0.03445
# 7      37528 / 5787630. =  0.00648
# 5      14813 / 5787630. =  0.00256
# 6        292 / 5787630. =  0.00005

print (len(df_merged.paper_UT.unique())  # 156558   only articles!!  (no reviews, commentaries,corrections,...)
       
       
       
       

####### get overall avg. and quantiles      

list_quantiles_cell=[.25,.5,.75]
values_quantiles=list(plos_df['total_refs'].quantile(list_quantiles_cell))     
print("for plos_df\n       avg # references included:",plos_df.total_refs.mean(),  "  STD:", plos_df.total_refs.std(), "   25-50-75:",values_quantiles)




df_references = df_merged.drop_duplicates(subset=['reference_UT'])
#print (df_references.shape)


list_quantiles_cell=[.25,.5,.75]
values_quantiles_cit=list(df_references['cite_count'].quantile(list_quantiles_cell))     
values_quantiles_age=list(df_references['diff_year_plos_ref'].quantile(list_quantiles_cell)) 
print("for df_references\n       avg # citations:",df_references.cite_count.mean(),  "  STD:", df_references.cite_count.std(),  "  25-50-75:",values_quantiles_cit,"\n       avg age diff:",df_references.diff_year_plos_ref.mean(),  "  STD:", df_references.diff_year_plos_ref.std(),  "  25-50-75:",values_quantiles_age)





list_quantiles_cell=[.25,.5,.75]
values_quantiles=list(df_merged['cite_count'].quantile(list_quantiles_cell))#sorted(list(df_select[v1_string].quantile(list_quantiles_cell).to_dict().items()))      
print("for all df_merged records\n       avg # citations:",df_merged.cite_count.mean(),  "  STD:", df_merged.cite_count.std(),  "   25-50-75:",values_quantiles)



                     
       
       
### i look at the groups       
df_merged.plos_j1.value_counts() 

# PLOS ONE       5368044
# PLOS GENET      119685
# PLO NE TR D      95315
# PLOS PATHOG      83451
# PLOS COMPUT      65177
# PLOS BIOL        44186
# PLOS MED         11776
# Name: plos_j1, dtype: int64
       
  
       
df_merged.plos_field.value_counts()  
# ['D RO MULTIDISCIPLINARY SCIENCES']                                                                                                       3779069
# ['D CU BIOLOGY']                                                                                                                           880492
# ['D RO MULTIDISCIPLINARY SCIENCES', 'D CU BIOLOGY']                                                                                        708479
# ['D KM GENETICS & HEREDITY']                                                                                                               119685
# ['D YU TROPICAL MEDICINE', 'D TI PARASITOLOGY']                                                                                             95315
# ['D ZE VIROLOGY', 'D QU MICROBIOLOGY', 'D TI PARASITOLOGY']                                                                                 83451
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY']                                                          65072
# ['D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY', 'D CU BIOLOGY']                                                                                   44186
# ['D PY MEDICINE, GENERAL & INTERNAL']                                                                                                       11776
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY', 'D PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS']        105


       
df_merged.groupby(['plos_field','plos_j1']).size()#.value_counts()  

# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY', 'D PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS']  PLOS COMPUT        105
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY']                                                      PLOS COMPUT      65072
# ['D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY', 'D CU BIOLOGY']                                                                               PLOS BIOL        44186
# ['D CU BIOLOGY']                                                                                                                        PLOS ONE        880494
# ['D KM GENETICS & HEREDITY']                                                                                                            PLOS GENET      119685
# ['D PY MEDICINE, GENERAL & INTERNAL']                                                                                                   PLOS MED         11776
# ['D RO MULTIDISCIPLINARY SCIENCES', 'D CU BIOLOGY']                                                                                     PLOS ONE        708480
# ['D RO MULTIDISCIPLINARY SCIENCES']                                                                                                     PLOS ONE       3779070
# ['D YU TROPICAL MEDICINE', 'D TI PARASITOLOGY']                                                                                         PLO NE TR D      95315
# ['D ZE VIROLOGY', 'D QU MICROBIOLOGY', 'D TI PARASITOLOGY']                                                                             PLOS PATHOG      83451

In [10]:
#print (select_data_for_plotting.__doc__)

#### Auxiliary function to select only a subset of the data, according to a number of parameters

In [8]:

def select_data_for_plotting(df_merged, v1_string, years, string_filtering_x, string_references_age, string_isolated_ref, string_self_ref, string_code_categ, string_journal, string_plos_field):
    """
    This function selects a subset of the total dataset acording to a set of parameters.

            

    Parameters
    ----------
    df_merged : pandas dataframe
        Original dataframe to select from
        
    v1_string : str
        Main variable 
        
    years : list of int
        List of selected years
        
    string_filtering_x : str
        Variable for heatmap plot
        
    string_references_age : str
        Selected 'young' or 'old' or 'all' references for the analysis
        
    string_isolated_ref : str
        Selected isolated (1) or group (0) or 'all' references for the analysis
        
    string_self_ref : str
         Selected self references (1) or not self references (0) or 'all' references for the analysis
         
    string_code_categ : str
        Selected PLos category: from 0 to 10 or multiple ones
        
    string_journal : int
        Selected PLoS journal
        
    string_plos_field : str
        Selected PLoS field


    Returns
    -------
    dataframe
        Selected subset of rows from the original dataset (all columns).

    """

    
    
    
    
    
    
    print ("original size:",df_merged.shape)



    ##### preselection by plos publication year
    print (years)
    preselection_df = df_merged[df_merged['plos_pub_year'].isin(years)]  
    print ("size of preselection1 (by plos years):",preselection_df.shape)





    #### i remove self-citations
    if (string_self_ref==0) or  ( string_self_ref == 1 ): 
        preselection_df = preselection_df[preselection_df['self_citation']== string_self_ref ]  
        if string_self_ref ==0:
            string_self_ref = ", no self-cit"
        elif string_self_ref ==1:
            string_self_ref = ", only self-cit"





    ######### preselection by isolated or group references:
    if (string_isolated_ref==0) or  ( string_isolated_ref == 1 ): 
        preselection_df0 = preselection_df[preselection_df['isolated_citation']== string_isolated_ref ]  

        if string_isolated_ref ==0:
            string_isolated_ref = ", group ref"
        elif string_isolated_ref ==1:
            string_isolated_ref = ", isolated ref"
    else:    
        preselection_df0 = preselection_df   
        print ("size of preselection1 (by isolated/group ref):",preselection_df0.shape, string_isolated_ref)








    ######### preselection by plos ONE subject category:
    if string_code_categ=="": 
        preselection_df111 = preselection_df0
    else:    
        if " " not in string_code_categ:  # to include one single category
            preselection_df111 = preselection_df0[preselection_df0['categ_codes'].str.contains(string_code_categ)]        
            string_code_categ = " "+dict_code_categ[string_code_categ]  

        else:  # if multiple codes-categories
            list_codes = string_code_categ.split(" ")
            print (list_codes)

            if len(list_codes) >= 2:              
                preselection_df111 = preselection_df0[ preselection_df0['categ_codes'].str.contains('|'.join(list_codes)) ]  # to look for partial matches from a list of strings!!!!!


            string_code_categ = "" 
            for code in list_codes:
                string_code_categ += "-"+dict_code_categ[code] 


        print (" size of preselection (by plos ONE subject category):",preselection_df111.shape, string_code_categ)








    ######### preselection by plos journal:
    if string_journal=="": 
        preselection_df1 = preselection_df111
    else:    
        preselection_df1 = preselection_df111[preselection_df111['plos_j1']== string_journal ]  
    print (" size of preselection2 (by plos journal):",preselection_df1.shape, string_journal)







    ######### preselection by plos field:
    if string_plos_field=="": 
        preselection_df2 = preselection_df1
    else:    
        preselection_df2 = preselection_df1[preselection_df1['plos_field']== string_plos_field ]  
    print (" size of preselection2 (by plos field):",preselection_df2.shape, string_plos_field)


    preselection_df3 = preselection_df2

    if v1_string ==  'cite_count'  or       v1_string ==  'log_num_cit_ref'   or v1_string == 'log2_num_cit_ref':

        string_age_selv1_stringection=''

        ##### preselection only young/old references:        
        if string_references_age == "young":
            time_window = 1
            string_age_selection="only young references from >="+ str((min(years)-time_window))
            preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] >= (min(years)-time_window) ]   
            print ("  size of preselection3 (only young references):",preselection_df3.shape, string_age_selection)

        elif string_references_age == "old":
            time_window = 10
            string_age_selection="only old references from <="+str((min(years)-time_window))
            preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] <= (min(years)-time_window) ]   
            print ("  size of preselection3 (only young references):",preselection_df3.shape,string_age_selection )

        else:
            string_age_selection="young&old"       
            print ("  No preselection by age of references:",preselection_df3.shape )




  

    
    
    
    
    
    
    return preselection_df3

## FIGURE: Multiplot for barplots of total number of records and total number of papers per journal

#### Select the the subset of data for plotting:

In [ ]:
    
v1_string =  'cite_count'

years=[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]   #### SELECT THE YEARS TO BE INCLUDED

string_filtering_x = 'paper_cite_count'   

string_references_age = ""   # AGE OF REFERENCES TO BE INCLUDED:  young     old   

string_isolated_ref = ""   # WHETHER OR NOT TO INCLUDE ISOLATED REFERENCES:   # 0  or 1 (or empty string, to include all ref)

string_self_ref =0    #  WHETHER OR NOT TO INCLUDE SELF-REFERENCES:  #     0  or 1 (or empty string, to include all ref)


string_code_categ="" #  plos ONE categories:  the codes are strings (0  TO 10), not integers. if i want to include multiple subjects:  "1 2 8"

string_journal=""   #  plos journals 

string_plos_field=""  #  WoS subject categories. 



preselection_df3 = select_data_for_plotting(df_merged, v1_string, years, string_filtering_x, string_references_age, string_isolated_ref, string_self_ref, string_code_categ, string_journal, string_plos_field)




print ("\nTot # records included:",len(preselection_df3),"   # number of plos papers:",len(preselection_df3.paper_UT.unique()), "   # unique ref:", len(preselection_df3.reference_UT.unique()),'\n')

####################################



    
    

#### I create the (empty) multiplot to be populated with the different subplots:

In [ ]:

fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.001, horizontal_spacing=0.001)
    
    
list_colors = ["#0000FF","#FF4040","#4EEE94","#87CEFA","#FFA500","#EE82EE","#8B8B83"]
list_journals = ['PLOS ONE', 'PLOS GENET', 'PLO NE TR D','PLOS PATHOG', 'PLOS COMPUT', 'PLOS BIOL',  'PLOS MED']



data = []




#### I populate the multiplot:

In [ ]:
  
### FOR THE NUMBER OF PAPERS
cont = 0
for journal in list_journals:
    df_selection = preselection_df3[preselection_df3['plos_j1'] == journal]        
    
    trace1 = go.Bar(
        x=['# papers'],
        y=[ len(df_selection.paper_UT.unique())],
        marker=dict( color=list_colors[cont]),
        name=journal.replace('PLO NE TR D','Neglected<br>  Tropical Diseases').replace('PLOS BIOL','Biology').replace('PLOS COMPUT','Computational<br>  Biology').replace('PLOS GENET','Genetics').replace('PLOS MED','Medicine').replace('PLOS PATHOG','Pathology'),
        showlegend=True
    )
   
    fig.append_trace(trace1, 1, 1)
    cont +=1


    
### FOR THE NUMBER OF RECORDS    
cont = 0    
for journal in list_journals:
    df_selection = preselection_df3[preselection_df3['plos_j1'] == journal]
    
    
    trace2 = go.Bar(
        x=['# records'],   
        y=[len(df_selection)],
        marker=dict( color=list_colors[cont]),
        showlegend=False,
        xaxis='x2',
        yaxis='y2'
    )
    fig.append_trace(trace2, 1, 2)
    cont  +=1

 





#### I customize the layout:

In [ ]:

font_gral=55 # 20 if i wanna see it on the browser, 40 if i care about the png output


fig['layout'].update( legend=dict(x=1.2, y=1.0), margin=go.Margin( l=250))#, b=150),)# r=50, b=150,  #t=100, # pad=4 ), )
fig['layout'].update(barmode='stack')
#fig['layout']['yaxis'].update(type='log')
#fig['layout']['yaxis'].update(range=[0, 5.3])  ### ojo!! rango logaritmico!
#fig['layout']['yaxis2'].update(type='log')
#fig['layout']['yaxis'].update(range=[0, 6.84])  ### ojo!! rango logaritmico!
fig['layout']['yaxis2'].update(side='right')
fig['layout']['yaxis'].update(title='Count')
fig['layout']['font']['size'] = font_gral







# axis
fig['layout']['xaxis']['tickfont']['size']  = font_gral + 10
fig['layout']['xaxis2']['tickfont']['size'] = font_gral + 10
fig['layout']['yaxis']['tickfont']['size']  = font_gral + 10
fig['layout']['yaxis2']['tickfont']['size'] = font_gral + 10
fig['layout']['xaxis'].update(domain=[0, 0.45]) # this is to force the relative location of the two panels with respect to each other
fig['layout']['yaxis'].update(domain=[0, 1])
fig['layout']['xaxis2'].update( domain=[0.55, 1])
fig['layout']['yaxis2'].update(domain=[0, 1],  anchor='x2')




#### Plot:

In [ ]:

offline.plot(fig, auto_open=True, image = 'png', image_filename="fig1c" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/fig1c', validate=True)




This would be the result

<img src="files/Barplot_Number_papers_number_records.png">

FIGURE CAPTION: number of papers and number of records in our dataset for each one of the different PLoS journals, and it includes all publication years together.

## FIGURE:  Heatmap plot for median age of referenceds used, by impact category of the citing paper, and by paper section.  
## Next, I'll also run pairwise statistical comparisons between all pairs of cells in a given heatmap, to determine whether the observed differences are significant.

#### I select the subset of data for plotting:

In [ ]:
    
v1_string =  'diff_year_plos_ref' 
string_references_age = ""   # AGE OF REFERENCES TO BE INCLUDED:  young     old   
      
    
if v1_string ==  'cite_count'  :
    colorbar_string = 'Citations'
    fig_colorscale = "Reds"
    fig_font_colors = ['#ff0000', '#ffece6']
else:
    colorbar_string = 'Age [yr]' 
    fig_colorscale = "Greens"
    fig_font_colors = ['#205803', '#dcf0d2'] 
  




years = [2009]   #### SELECT THE YEARS TO BE INCLUDED

string_filtering_x = 'paper_cite_count'   

 

string_isolated_ref = ""   # WHETHER OR NOT TO INCLUDE ISOLATED REFERENCES:   # 0  or 1 (or empty string, to include all ref)

string_self_ref =0    #  WHETHER OR NOT TO INCLUDE SELF-REFERENCES:  #     0  or 1 (or empty string, to include all ref)


string_code_categ="" #  plos ONE categories:  the codes are strings (0  TO 10), not integers. if i want to include multiple subjects:  "1 2 8"

string_journal=""   #  plos journals 

string_plos_field=""  #  WoS subject categories. 




preselection_df3 = select_data_for_plotting(df_merged, v1_string, years, string_filtering_x, string_references_age, string_isolated_ref, string_self_ref, string_code_categ, string_journal, string_plos_field)
print ("\nTot # records included:",len(preselection_df3),"   # number of plos papers:",len(preselection_df3.paper_UT.unique()), "   # unique ref:", len(preselection_df3.reference_UT.unique()),'\n')

####################################







dict_group_subset_data={}
dict_group_quantiles_size={}

 
  
######## i get the bins number of citation of the plos papers: i want the same bins for all papers (so i calculate them before separating into sections but after all the preselections)

list_q=[0.3,0.6,.9,.99,1]    # for the percentile sections for number of citations of the PLOS papers
   
quantiles=sorted(list(preselection_df3[string_filtering_x].quantile(list_q).to_dict().items())) #mean 10.68 


lista_bins_plos_citations=[]
old_value=0
for item in quantiles:
    try:
        pair=[old_value, int(item[1])]    
    except:  # if it is a nan:
        pair=[old_value, item[1]]

    lista_bins_plos_citations.append(pair)

    try:
        old_value = int(item[1])
    except:
        old_value = item[1]

print (lista_bins_plos_citations)

################################################




#### I create the grouping of data corresponding to the different cells of the plot:

In [ ]:
  
lista_titulos_sets = []
lista_sections = ["Introduction","Methods","Results","Discussion"]

cont=0
for item in lista_bins_plos_citations:

    minimo = item[0]
    maximo = item[1]


    preselection_df4 = preselection_df3[(preselection_df3[string_filtering_x] >= minimo)  &  (preselection_df3[string_filtering_x] < maximo)]  

    x1_All = list(preselection_df4[v1_string])



    for string_section in lista_sections:


        ##### preselection to include only occurrences in a section of the paper
        if  string_section == "Introduction":
            section=0
        elif  string_section == "Methods":
            section=1
        elif  string_section == "Results":
            section=2
        elif  string_section == "Discussion":
            section=3



        df_select = preselection_df4[preselection_df4['regex_sect_index']== section]   
        

        x1 = list(df_select[v1_string])       





        if cont ==0:            
            group=string_section+" Bottom" 
        elif cont ==1:            
                   
            group=string_section+" Typical"       
        elif cont==2:
                 
             group=string_section+" Good"    
        elif cont==3: 
               
            group=string_section+" High"
        elif cont==4:
                 group=string_section+" Top"
            
            
        lista_titulos_sets.append(group)    





        ######### i get also quantiles for each cell:    
        list_quantiles_cell=[.25,.5,.75]

        values_quantiles=list(df_select[v1_string].quantile(list_quantiles_cell))#sorted(list(df_select[v1_string].quantile(list_quantiles_cell).to_dict().items()))      

        tupla=values_quantiles + [len(x1)]

        dict_group_quantiles_size[group] = tupla

        dict_group_subset_data[group]=x1



    cont +=1





################ i also add the median values for the section across all data in the preselection
for string_section in lista_sections:


    if  string_section == "Introduction":
        section=0
    elif  string_section == "Methods":
        section=1
    elif  string_section == "Results":
        section=2
    elif  string_section == "Discussion":
        section=3


    df_select = preselection_df3[preselection_df3['regex_sect_index']== section]   

    list_quantiles_cell=[.25,.5,.75]
    values_quantiles=list(df_select[v1_string].quantile(list_quantiles_cell))  
    tupla=values_quantiles + [len(df_select)]



########  I create the x, y, z lists of values for the heatmap

lista_y=lista_sections

lista_bin_names=[" Bottom"," Typical"," Good"," High"," Top"]

lista_x=lista_bin_names

lista_z25=[]
lista_z50=[]
lista_z75=[]
lista_z_sizes=[]

for x_value in lista_x:    
    aux_lista25=[]
    aux_lista50=[]
    aux_lista75=[]
    aux_lista_sizes=[]

    for y_value in lista_y:       

        llave=y_value+x_value

        try:
            value=int(dict_group_quantiles_size[llave][0])
        except:  # if it is a nan:
            value=dict_group_quantiles_size[llave][0]
        aux_lista25.append(value)




        try:
            value=int(dict_group_quantiles_size[llave][1])
        except:  # if it is a nan:
            value=dict_group_quantiles_size[llave][1]
        aux_lista50.append(value)




        try:
            value=int(dict_group_quantiles_size[llave][2])
        except:  # if it is a nan:
            value=dict_group_quantiles_size[llave][2]
        aux_lista75.append(value)




        value_size=dict_group_quantiles_size[llave][3]
        aux_lista_sizes.append(value_size)


        #print (y_value," ",x_value, value, value_size)
    lista_z25.append(aux_lista25)
    lista_z50.append(aux_lista50)
    lista_z75.append(aux_lista75)




    lista_z_sizes.append(aux_lista_sizes)



    
######## I get a customized text for each cell
lista_text_z=[]
for i in range(len(lista_z_sizes)):
    aux=[]
    for j in range(len(lista_z_sizes[0])):        
        value=str(lista_z25[i][j])+"-<b>"+str(lista_z50[i][j])+"</b>-"+str(lista_z75[i][j])+"<br>("+str(format(lista_z_sizes[i][j], ',d'))+")"           

        aux.append(value)
        
    lista_text_z.append(aux)



    
    


####  color selection and titles for the figure:

In [ ]:

top_space = 150
if v1_string ==  'cite_count'  :
    colorbar_string = 'Citations'
    if string_references_age == "old" :
        colorbar_string = ''
else:
    colorbar_string = 'Age [yr]'
    top_space = 100
    text_abc = '(b)'

    
fig_font_colors=''


factor_color_rescale =.6  

fig_colorscale=[[0.0*factor_color_rescale, '#ffffff'],\
                       [0.1*factor_color_rescale, '#d9f2d9'],\
                       [0.2*factor_color_rescale, '#c6ecc6'],\
                       [0.3*factor_color_rescale, '#b3e6b3'],\
                       [0.4*factor_color_rescale, '#8cd98c'], \
                       [0.5*factor_color_rescale, '#66cc66'], \
                       [0.6*factor_color_rescale, '#53c653'], \
                       [0.7*factor_color_rescale, '#40bf40'],\
                       [0.75*factor_color_rescale, '#39ac39'],\
                       [0.8*factor_color_rescale, '#339933'],\
                       [0.85*factor_color_rescale, '#2d862d'],\
                       [0.9*factor_color_rescale, '#267326'],\
                      [1.0, '#000000']]



fig_font_colors = ['#205803', '#dcf0d2']      # same for the annotation of the boxes (to make sure they are readable)
fig_filename = '../plots/annotated-heatmap_median_age_difference_plos_publ_year_vs_references_for_sections_and_subsect_by_citations_of_plos'












#### I create the figure:

In [ ]:

fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names, annotation_text=lista_text_z, colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True, colorbar=dict(title=colorbar_string, titleside='right' ),)#, reversescale=True)


#### Layout:

In [ ]:

fig.layout.title = ""# fig_title_plot

fig['layout']['xaxis']['side'] = 'bottom'
fig.layout.xaxis.update({'title': 'Section'})


fig.layout.yaxis.update({'title': 'Impact Group'})
if v1_string ==  'cite_count'  :
    if string_references_age == "young":  
        fig.layout.yaxis.update({'title': ''})

font_gral=25  # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral

      
if v1_string ==  'cite_count'  :
    if string_references_age == "young":  
        #fig.layout.update({'title': '$d, r \\text{ (solar radius)}$'})
        fig['layout']['title'] = "Young references"
    elif string_references_age == "old":  
        fig.layout.update({'title': 'Old references'})

    fig.layout.update({'font': dict(size=25)})


font_gral=55  # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral


# axis

fig['layout']['xaxis']['tickangle'] = 0
fig['layout']['yaxis']['tickangle'] = -90
fig['layout']['xaxis']['titlefont']['size'] = font_gral + 20
fig['layout']['yaxis']['titlefont']['size'] = font_gral
fig['layout']['xaxis']['tickfont']['size'] = font_gral -7 
fig['layout']['yaxis']['tickfont']['size'] = font_gral -15
fig['layout']['margin']=dict(
        l=200,
       # r=50,
        b=150,
        t=top_space,
        pad=15
    )



#### Plot:

In [ ]:
offline.plot(fig, auto_open=True, image = 'png', image_filename=fig_filename ,image_width=2000, image_height=1200, filename=fig_filename+'.html', validate=True)



In [ ]:
This would be the result:

<img src="files/Heatmap_plot_age_of_references_by_section_and_paper_impact_group.png">

FIGURE CAPTION (THIS RESULTS ARE STILL UNPUBLISHED, PLEASE KEEP IT CONFIDENTIAL): The references used in the Methods section of  PLoS papers are the oldest, while the ones used in the Discussion section are the youngest. We also observe how the higher the (future) impact of the citing paper, the youngers the references it uses, accross all paper sections.

##  Next, I need to run tests for the statistical comparison of all pairs of sub-sets (cells) displayed in the previous heatmap plot. Given that the original heatmap plot has 4x5 = 20 cells, that means I have to run N(N-1)/2  pairwise comparisons (to create and structure the plot more easily, I will repeat and replot the comparisons i-j  and j-i).
  

#### Select what statistical test to run: KS (Kolmogorov–Smirnov) to test if the distributions are different or  MW (mann whitney u test) for testing just whether the medians are different from each other


In [13]:
test = "MW"  # MW or KS

#### I apply the bonferroni correction, so my new p-value (p_value_threshold)  required for significance is: old_p_value /Number of comparisons = 0.001 / (20*19/2)     


In [ ]:

    
    
list_keys_macro = ['Introduction Top',    'Methods Top',    'Results Top',   'Discussion Top', \
                   'Introduction High',   'Methods High',  'Results High',  'Discussion High', \
                   'Introduction Good',   'Methods Good',   'Results Good',  'Discussion Good', \
                   'Introduction Typical', 'Methods Typical', 'Results Typical', 'Discussion Typical', \
                   'Introduction Bottom',  'Methods Bottom',  'Results Bottom', 'Discussion Bottom']
    
      
list_keys_heatmap = ['Introduction Bottom', 'Methods Bottom',  'Results Bottom', 'Discussion Bottom',\
                     'Introduction Typical', 'Methods Typical', 'Results Typical', 'Discussion Typical',\
                     'Introduction Good',   'Methods Good',   'Results Good',  'Discussion Good', \
                     'Introduction High',   'Methods High',  'Results High',  'Discussion High', \
                     'Introduction Top',   'Methods Top',    'Results Top',   'Discussion Top'  ]



    
   

lista_indeces = [[1,1],[1,2],[1,3],[1,4],\
                 [2,1],[2,2],[2,3],[2,4],\
                 [3,1],[3,2],[3,3],[3,4],\
                 [4,1],[4,2],[4,3],[4,4],\
                 [5,1],[5,2],[5,3],[5,4]]



threshold_zero = 0.0001 / (float(len(list_keys_macro))*float(len(list_keys_macro)-1)/2.)    # to round up to zero the very small p_values



#### I calculate the p-values corresponding to each pairwise comparison between cells, and i directly mark them as significant / non-significant (after correction)

In [ ]:

lista_tot_datos=[]

total_cont =0
for i in list_keys_macro:
    lista_listas=[]
    aux_lista=[]
    cont=1
   
    for j in list_keys_heatmap:
             
        set1 = dict_group_subset_data[i]
        set2 = dict_group_subset_data[j]
        
        if test == "KS":
            p_value = stats.ks_2samp(set1, set2)[1] 
        elif test == "MW":
            p_value = stats.mannwhitneyu(set1, set2,  alternative='two-sided')[1]  
        
        
        if p_value <= threshold_zero:  #i round up to zero the very small p_values
                p_value =0.
                
        else:
            p_value = .5  ### I ONLY CARE ABOUT WHETHER IT IS SIGNIFICANT OR NOT, I DONT CARE ABOUT THE EXACT P-VALUE ONCE IT IS NOT SIGNIFICANT
                
        if i == j:  # i single out manually the self comparison  (to color it differently later)
            p_value=1.001  
            
            
            
        aux_lista.append(p_value)
        
        cont +=1
        
        if cont > tot_cols:
            lista_listas.append(aux_lista)                  
            aux_lista=[]
            cont=1
            
    
    
    total_cont +=1
    lista_tot_datos.append(lista_listas)
    
    


 
#### I create an empty plot and I populate it with the different pairwise comparisons



In [ ]:

fig_colorscale=  [ [0., '#0059b3'], [.5,'#c7dcf1'], [1.,'#bdbdbd']] #  0 or anything significant: blue,   .5 or anithing NON signif: light-blue,     1: grey
lista_bin_names = ["Bottom","Typical","Good","High","Top"]
lista_sections  =  ['Intro', 'Methods', 'Results', 'Discussion']


tot_rows = 5
tot_cols = 4  
    
      


fig_macro = None
fig_macro = tools.make_subplots(rows=tot_rows, cols=tot_cols, shared_xaxes=True, shared_yaxes=True, vertical_spacing = 0.01, horizontal_spacing = 0.01,   )


for i in range(len(lista_tot_datos)):
    datos = lista_tot_datos[i]
    
    cont_rows = lista_indeces[i][0]
    cont_cols = lista_indeces[i][1]
    
    trace1 = go.Heatmap(z=datos,
                       x=lista_sections,
                       y=lista_bin_names,                        
                       colorscale = fig_colorscale,
                       showscale=False,)
                        #reversescale=True, )#    )

   



    
    # i add each individual plot
    fig_macro.append_trace(trace1, cont_rows, cont_cols)


 

#### Layout:

In [ ]:

fontsize=32 
fig_macro['layout']['font'].update({'size': fontsize})


lista_bin_names = ["Bottom","Typical","Good","High","Top"]
lista_sections  =  ['Introduction', 'Methods', 'Results', 'Discussion']


# AXIS
fig_macro['layout']['xaxis1'].update(title=lista_sections[0])  
fig_macro['layout']['xaxis2'].update(title=lista_sections[1])
fig_macro['layout']['xaxis3'].update(title=lista_sections[2])
fig_macro['layout']['xaxis4'].update(title=lista_sections[3])


fig_macro['layout']['yaxis1'].update(title=lista_bin_names[4])   
fig_macro['layout']['yaxis2'].update(title=lista_bin_names[3])
fig_macro['layout']['yaxis3'].update(title=lista_bin_names[2])
fig_macro['layout']['yaxis4'].update(title=lista_bin_names[1])
fig_macro['layout']['yaxis5'].update(title=lista_bin_names[0])






#### Plot:

In [ ]:
offline.plot(fig_macro, auto_open=True, image = 'png', image_filename='multiplot_comparisons' ,image_width=3000, image_height=2200,
              filename='../plots/multiplot_comparisons.html', validate=True)



In [ ]:
This would be the result:

<img src="files/multiplot_pairwise_comparisons_WM_test_for_age_of_references.png">

FIGURE CAPTION (THIS RESULTS ARE UNPUBLISHED YET, PLEASE KEEP IT CONFIDENTIAL): We display the results from all pairwise comparison between cells from the previous figure (the GREEN one). 
Dark blue represents significantly different (after correction for multiple testing), light blue corresponds to non-significant, and grey marks a self-comparison. Most of the pairwise comparisons between cells are statistically significant.

## Implementation of a null model to compare the actual vs expected usage of top and bottom references by top and bottom PLoS papers.
## This randomization scheme controls for plos subject field and PLoS publication year, and it also preserves clusters of references used together in a section of a given paper.   
## This code does the randomization procces for the selected subset of data (Niter independent randomizations) and plots actual vs expected values, as well as provides z-scores.

#### Auxiliary function needed to implement the randomization scheme that preserves groups/clusters of references that are cited together in  a given paper


In [ ]:
def get_list_lists_references(preselection_df3):

    """
    This function provides a list of lists of references used by all PLOS papers in the selected sub set of data, 
    preserving those references that appear together in a cluster in a section of a given paper
            

    Parameters
    ----------
    preselection_df3 : pandas dataframe
        


    Returns
    -------
    list of lists with the IDs of the references
        

    """

    
   


    ####  NOTE: preselection_df3 only includes one instance of paper_UT-ref_UT  (the first occurrence in each paper), and i sort it too:
    preselection_df3.sort_values(by=['paper_UT','reference_UT'], inplace=True)


    distance_threshold = 5  # characters tops to separate group ref

    list_lists_all_ref = []  # WITH STRUCTURE
    lista_ref = []           # WITHOUT STRUCTURE
    cont = 0
    for paper_UT, group_df in preselection_df3.groupby(['paper_UT']):  #### OJO!!!! THIS LOOP IS WAY FASTER THAN DOING:  for   paper_UT in list_paper_UT    !!!!    

        group_df.sort_values(by=['regex_sect_index','sect_char_pos','reference_UT'],inplace = True)  # i sort the reference of a paper by section first, then by location within the section



        ### first i take care of the isolated references: 
        group_df1= group_df[group_df['isolated_citation'] ==1]        
        for index, row in group_df1.iterrows():  

            ref_UT = row['reference_UT']
            aux = [ref_UT]
            lista_ref.append(ref_UT)   
            list_lists_all_ref.append(aux)




        #### then i take care of the group references:
        group_df0= group_df[group_df['isolated_citation'] == 0]  
        previous_position = 0
        list_group_ref = []
        for index, row in group_df0.iterrows():

            ref_UT = row['reference_UT']
            lista_ref.append(ref_UT)   # list without structure (for comparison reasons)

            position = row['sect_char_pos']

            if previous_position == 0:  # for the very first entry                
                list_group_ref.append(ref_UT)


            else:  # for all other entries
                if (position - previous_position) <= distance_threshold  :   # if the current ref is close to the previous one
                     list_group_ref.append(ref_UT)
                else: 
                    list_lists_all_ref.append(list_group_ref)
                    list_group_ref = []
                    list_group_ref.append(ref_UT)




            previous_position = position

        list_lists_all_ref.append(list_group_ref)  # i need this for the final group/isolated one!!!

        cont +=1   







    #### i flatten out the list of lists for comparison purposes

    flat_list = []    
    for sublist in list_lists_all_ref:
        for item in sublist:
            flat_list.append(item)

    print ("list_lists created:", len(list_lists_all_ref), ' without structure:', len(lista_ref), "   flat_list:", len(flat_list))


   

    return list_lists_all_ref
    

#### Set the values of the parameters for the selection of the data, as well and number of iterations for the randomization:

In [16]:


Niter=1000   ##### number of iterations of the randomization process

years=[2010]  # controlling by year
    
    

    
########### select the subset of data for plotting:
v1_string = 'regex_sect_index'
       

string_filtering_x = 'paper_cite_count'   

string_references_age = "all"   # AGE OF REFERENCES TO BE INCLUDED:  young     old   

string_isolated_ref = ""   # WHETHER OR NOT TO INCLUDE ISOLATED REFERENCES:   # 0  or 1 (or empty string, to include all ref)

string_self_ref =0    #  WHETHER OR NOT TO INCLUDE SELF-REFERENCES:  #     0  or 1 (or empty string, to include all ref)


string_code_categ="" #  plos ONE categories:  the codes are strings (0  TO 10), not integers. if i want to include multiple subjects:  "1 2 8"

string_journal=""   #  plos journals 

string_plos_field=""  #  WoS subject categories. 


### PLoS subject category
string_code_categ = '0' #,'1'#,'2','3','4','5','6','7','8','9','10']  # I control by PLOS field: i run the randomizations independently for each one

######################3




preselection_df3 = select_data_for_plotting(df_merged, v1_string, years, string_filtering_x, string_references_age, string_isolated_ref, string_self_ref, string_code_categ, string_journal, string_plos_field) 



#### for the randomization, i only count each reference once per paper!!
preselection_df3 = preselection_df3.drop_duplicates(subset=['paper_UT', 'reference_UT'])






N_plos = len(preselection_df3.paper_UT.unique())         
N_ref = len(preselection_df3.reference_UT.unique()) 
N_all = len(preselection_df3)
print ("     N plos:", N_plos,"  N  ref:",N_ref, " N records:", N_all)        












#### This randomization will compare top vs bottom categories of either papers or references ('top' and 'bottom' defined as the percentiles of number of citations accrued), so I need to get the corresponding data for the percentiles in both cases.
#### First, I get the data corresponding to the percentiles for the PLoS papers.

In [ ]:


############## i define quantiles for plos papers and for references ("TOP" and "BOTTOM") 
list_q_plos=[.1,.9,1]
list_q_ref=[.1,.9,1]



df_for_quantiles_plos = preselection_df3.drop_duplicates(subset=['paper_UT'])  

quantiles=sorted(list(df_for_quantiles_plos['paper_cite_count'].quantile(list_q_plos).to_dict().items())) 
print ("\n\ncitation bins for the selected plos:", list_q_plos) 

lista_bins_plos=[]
old_value=0
for item in quantiles:
    pair=[old_value, int(item[1])]
    lista_bins_plos.append(pair)
    old_value = int(item[1])   

print ("\nbins for PLOS papers:")

cont = 0
dict_bin_list_plos_UT={}
for item in lista_bins_plos:

    minimo = item[0]
    maximo = item[1]   

    df_select = preselection_df3[(preselection_df3['paper_cite_count'] >= minimo)  &  (preselection_df3['paper_cite_count'] < maximo)]
    llave=str(minimo)+"-"+str(maximo)
    dict_bin_list_plos_UT[llave]= list(df_select.paper_UT.unique())
    print (" ",llave, "  N:",len(list(df_select.reference_UT.unique())), "  avg # ref:",df_select.drop_duplicates(subset=['paper_UT']).total_refs.mean())
    max_key_plos=llave


    if cont ==0:
        min_key_plos = llave
    cont  +=1



#### Similartly, I get the data in the percentiles for references' citations 

In [ ]:

############## i define quantiles for references ("TOP" and "BOTTOM") 
list_q_ref=[.1,.9,1]





df_for_quantiles_ref = preselection_df3.drop_duplicates(subset=['reference_UT'])   # ojo!!! remember to remove REPETITIONS!!!!
quantiles=sorted(list(df_for_quantiles_ref['cite_count'].quantile(list_q_ref).to_dict().items())) #mean 10.68 
print ("\n\ncitation bins for the references in the selected plos:", list_q_ref,quantiles)    

lista_bins=[]
old_value=0
for item in quantiles:
    pair=[old_value, int(item[1])]
    lista_bins.append(pair)
    old_value = int(item[1])




print ("\nbins for refrences:")


cont = 0
dict_bin_list_ref_UT={}
for item in lista_bins:

    minimo = item[0]
    maximo = item[1]    

    df_select = preselection_df3[(preselection_df3['cite_count'] >= minimo)  &  (preselection_df3['cite_count'] < maximo)]
    llave=str(minimo)+"-"+str(maximo)
    dict_bin_list_ref_UT[llave]=list(df_select.reference_UT.unique())
    print (" ",llave, "N:",len(list(df_select.reference_UT.unique())), "  avg # ref:",df_select.drop_duplicates(subset=['reference_UT']).total_refs.mean())
    max_key_ref=llave

    if cont ==0:
        min_key_ref = llave
    cont  +=1





#### I create the list of IDs of top papers, top references, bottom papers and bottom references that I will be comparing:


In [ ]:


lista_top_plos = dict_bin_list_plos_UT[max_key_plos]
print ("\n\n# UTs top",(100-100*list_q_plos[-2]),"% plos:",len(lista_top_plos))

lista_top_ref=dict_bin_list_ref_UT[max_key_ref]
print ("# UTs top",(100-100*list_q_ref[-2]),"% ref:", len(lista_top_ref))


lista_bottom_plos = dict_bin_list_plos_UT[min_key_plos]
print ("# UTs bottom ",(100*list_q_plos[0]),"% plos:",len(lista_bottom_plos))

lista_bottom_ref=dict_bin_list_ref_UT[min_key_ref]
print ("# UTs bottom ",(100*list_q_ref[0]),"% ref:", len(lista_bottom_ref))

list_plos_in_year= list(preselection_df3.paper_UT.unique())
print ("Tot # records:",len(preselection_df3),", # plos:",len(list_plos_in_year))


####  I obtain at the usage of the top references  

In [ ]:


df_top_ref = preselection_df3[preselection_df3['reference_UT'].isin(lista_top_ref)]


df_top_ref_top_plos = df_top_ref[df_top_ref['paper_UT'].isin(lista_top_plos)]
df_top_ref_bottom_plos = df_top_ref[df_top_ref['paper_UT'].isin(lista_bottom_plos)]


usage_top_ref_top_plos = len(df_top_ref_top_plos)/float(len(df_top_ref))
usage_top_ref_bottom_plos = len(df_top_ref_bottom_plos)/float(len(df_top_ref))


print ("fraction of usage of top ref by ")
print ("  top",(100-100*list_q_plos[-2]),"% plos:",  usage_top_ref_top_plos)
print ("  bottom",(100*list_q_plos[0]),"% plos:",usage_top_ref_bottom_plos  )


##### I obtain at the usage of the bottom references  

In [ ]:
df_non_top_ref = preselection_df3[preselection_df3['reference_UT'].isin(lista_bottom_ref)]


df_non_top_ref_top_plos = df_non_top_ref[df_non_top_ref['paper_UT'].isin(lista_top_plos)]
df_non_top_ref_bottom_plos = df_non_top_ref[df_non_top_ref['paper_UT'].isin(lista_bottom_plos)]

usage_non_top_ref_top_plos = len(df_non_top_ref_top_plos)/float(len(df_non_top_ref))
usage_non_top_ref_bottom_plos = len(df_non_top_ref_bottom_plos)/float(len(df_non_top_ref))


print ("fraction of usage of non-top ref by ")
print ("  top",(100-100*list_q_plos[-2]),"% plos:", usage_non_top_ref_top_plos )
print ("  bottom",(100*list_q_plos[0]),"% plos:", usage_non_top_ref_bottom_plos )





#### Now I have the actual usage. 
#### Next, I enter the loop to canculate the expected values from my null model (usage of references by top and non top plos papers, from the randomized data)

In [ ]:

lista_usage_top_ref_by_top_plos_rand = []
lista_usage_top_ref_by_bottom_plos_rand = []

lista_usage_nontop_ref_by_top_plos_rand = []
lista_usage_nontop_ref_by_bottom_plos_rand = []





#### first i get the list of lists corresponding to the references used in the selected df, preserving the reference grouping or isolation:  
lista_lists_values = get_list_lists_references(preselection_df3)

print ("len list_lists_all_ref:",len(lista_lists_values), preselection_df3.shape)


for i in range(Niter):

    print (i)


    ########   new randomization scheme (controling for year, but also preserving groups of references cited together in a paper):   
    #### lista_values is created outside the Niter loop   (i only need to do it once per selected df)    



    random.shuffle(lista_lists_values)      
    ### to flat out a list of lists:   
    flat_list = []    
    for sublist in lista_lists_values:
        for item in sublist:
            flat_list.append(item)




    preselection_df3['randomized_ref_UT'] = flat_list   ### (this randomizes paper_UT, not reference_UT)








    ####### (RANDOMIZED)  i look at the usage of the top ref
    df_top_ref_rand = preselection_df3[preselection_df3['randomized_ref_UT'].isin(lista_top_ref)]

    df_top_ref_top_plos_rand = df_top_ref_rand[df_top_ref_rand['paper_UT'].isin(lista_top_plos)]
    df_top_ref_bottom_plos_rand = df_top_ref_rand[df_top_ref_rand['paper_UT'].isin(lista_bottom_plos)]


    usage_top_ref_top_plos_rand = len(df_top_ref_top_plos_rand)/float(len(df_top_ref_rand))
    usage_top_ref_bottom_plos_rand = len(df_top_ref_bottom_plos_rand)/float(len(df_top_ref_rand))


    lista_usage_top_ref_by_top_plos_rand.append(usage_top_ref_top_plos_rand)
    lista_usage_top_ref_by_bottom_plos_rand.append(usage_top_ref_bottom_plos_rand)






    #######  (RANDOMIZED) i look at the usage of the non-top ref            
    df_non_top_ref_rand = preselection_df3[preselection_df3['randomized_ref_UT'].isin(lista_bottom_ref)]        

    df_non_top_ref_top_plos_rand = df_non_top_ref_rand[df_non_top_ref_rand['paper_UT'].isin(lista_top_plos)]
    df_non_top_ref_bottom_plos_rand = df_non_top_ref_rand[df_non_top_ref_rand['paper_UT'].isin(lista_bottom_plos)]


    usage_non_top_ref_top_plos_rand = len(df_non_top_ref_top_plos_rand)/float(len(df_non_top_ref_rand))
    usage_non_top_ref_bottom_plos_rand = len(df_non_top_ref_bottom_plos_rand)/float(len(df_non_top_ref_rand))


    lista_usage_nontop_ref_by_top_plos_rand.append(usage_non_top_ref_top_plos_rand)
    lista_usage_nontop_ref_by_bottom_plos_rand.append(usage_non_top_ref_bottom_plos_rand)











print ("\n\n\n\navg randomized!!")
print("fraction of usage of top ref by")
print ("  top",(100-100*list_q_plos[-2]),"% plos:",  np.mean(lista_usage_top_ref_by_top_plos_rand) )   
print ("  bottom",(100*list_q_plos[0]),"% plos:",np.mean(lista_usage_top_ref_by_bottom_plos_rand)  )




print ("\n\navg randomized")
print ("fraction of usage of non-top ref by ")
print ("  top",(100-100*list_q_plos[-2]),"% plos:", np.mean(lista_usage_nontop_ref_by_top_plos_rand) )   
print ("  bottom",(100*list_q_plos[0]),"% plos:", np.mean(lista_usage_nontop_ref_by_bottom_plos_rand) ,"\n\n\n")








###  I get the corresponding z-scores from the comparisons actual vs. expected

In [ ]:

lista_bin_names = ['Bottom '+str(int(list_q_plos[0]*100))+'%<br>papers', 'Top '+str(int(100-100*list_q_plos[-2]))+'%<br>papers']

lista_for_top_ref = [ usage_top_ref_top_plos, usage_top_ref_bottom_plos]
lista_for_bottom_ref = [usage_non_top_ref_top_plos, usage_non_top_ref_bottom_plos]




lista_for_top_ref = [  usage_top_ref_bottom_plos, usage_top_ref_top_plos]
lista_for_bottom_ref = [usage_non_top_ref_bottom_plos, usage_non_top_ref_top_plos]





###### this is the null model 
lista_expectations_top_ref = [np.mean(lista_usage_top_ref_by_bottom_plos_rand), np.mean(lista_usage_top_ref_by_top_plos_rand)]  
lista_expectations_bottom_ref = [ np.mean(lista_usage_nontop_ref_by_bottom_plos_rand),np.mean(lista_usage_nontop_ref_by_top_plos_rand)] 


list_errors_top_ref = [2.*np.std(lista_usage_top_ref_by_bottom_plos_rand), 2.*np.std(lista_usage_top_ref_by_top_plos_rand) ] 
list_errors_bottom_ref = [2.*np.std(lista_usage_nontop_ref_by_bottom_plos_rand), 2.*np.std(lista_usage_nontop_ref_by_top_plos_rand) ] 


z_score_top_ref_by_top_plos = (usage_top_ref_top_plos - np.mean(lista_usage_top_ref_by_top_plos_rand))/np.std(lista_usage_top_ref_by_top_plos_rand)
z_score_nontop_ref_by_top_plos = (usage_non_top_ref_top_plos - np.mean(lista_usage_nontop_ref_by_top_plos_rand))/np.std(lista_usage_nontop_ref_by_top_plos_rand)

z_score_top_ref_by_bottom_plos = (usage_top_ref_bottom_plos - np.mean(lista_usage_top_ref_by_bottom_plos_rand))/np.std(lista_usage_top_ref_by_bottom_plos_rand)
z_score_nontop_ref_by_bottom_plos = (usage_non_top_ref_bottom_plos - np.mean(lista_usage_nontop_ref_by_bottom_plos_rand))/np.std(lista_usage_nontop_ref_by_bottom_plos_rand)




print ('zscore top ref cited  by top plos:', z_score_top_ref_by_top_plos)
print ('zscore bottom ref cited  by top plos:', z_score_nontop_ref_by_top_plos)

print ('zscore top ref cited  by bottom plos:', z_score_top_ref_by_bottom_plos)
print ('zscore bottom ref cited  by bottom plos:', z_score_nontop_ref_by_bottom_plos)


### FIGURE: plot the results from the randomization

#### Traces:

In [ ]:


title_string=''



trace1 = go.Bar(
    x=lista_bin_names,
    y=lista_for_top_ref,    
    marker=dict(
            color='#88419d',           
    ),

)



trace2 = go.Bar(
    x=lista_bin_names,
    y=lista_expectations_top_ref,
    error_y=dict(           
            array=list_errors_top_ref,
            thickness=5,
            visible=True
    ),
    marker=dict(
            color='#c994c7',         
    ),
)


trace3 = go.Bar(
    x=lista_bin_names,
    y=lista_for_bottom_ref,
      marker=dict(
            color='#225ea8',   
            ),
)


trace4 = go.Bar(
    x=lista_bin_names,
    y=lista_expectations_bottom_ref,
    error_y=dict(       
            array=list_errors_bottom_ref,#[0.5, 1, 2],
            thickness=5,
            visible=True
            ),
    marker=dict(
            color='#a6bddb',     
    ),
)



data = [trace1, trace2, trace3, trace4]





####  Layout:

In [ ]:


size_bar_name = 30
y_pos_bar_names = -.039
angle = -70
font_gral=30

layout = go.Layout(   
    title=title_string,
    xaxis = dict(
        side= 'top',
        range = [-.5,1.5],
       # showline =  True,
        #title= 'Plos Citation percentile'),
    ),
    yaxis = dict(
        title= 'Fraction of references cited',
        range = [-.07,0.17],
        tickvals=[0.0,0.05,0.1,0.15],
        #showline =  True,
         ),

    showlegend=False,
    bargroupgap=0.15,


    #### ADD ANNOTATIONS:
    annotations = [  
        # the four bars on the left
        dict(
          x = -.34,
          y = y_pos_bar_names,
          text = 'Top '+str(int(100-100*list_q_ref[-2]))+'%<br>references',
          textangle=angle,
            font = dict(size = size_bar_name ),
           ),    
        dict(
          x = -0.14,
          y = y_pos_bar_names,
          text = 'Null Model',
           textangle=angle,
            font = dict(size = size_bar_name ),
           ),
        dict(
          x = .08,
          y = y_pos_bar_names,
          showarrow = False,
          text = 'Bottom '+str(int(list_q_ref[0]*100))+'%<br>references', 
          textangle=angle,
            font = dict(size = size_bar_name ),
           ),
        dict(
          x = .28,
          y = y_pos_bar_names,
          text =  'Null Model',
          textangle=angle,
            font = dict(size = size_bar_name ),
           ),



        # the four bars on the right
        dict(  
          x = .68,
          y = y_pos_bar_names,
          text = 'Top '+str(int(100-100*list_q_ref[-2]))+'%<br>references',
          textangle=angle,
            font = dict(size = size_bar_name ),
           ),
        dict(
          x = .88,
          y = y_pos_bar_names,
          text = 'Null Model',
           textangle=angle,
            font = dict(size = size_bar_name ),
           ),
        dict(
          x = 1.08,
          y = y_pos_bar_names,
          text = 'Bottom '+str(int(list_q_ref[0]*100))+'%<br>references', 
          textangle=angle,
            font = dict(size = size_bar_name ),
           ),
        dict(
          x = 1.28,
          y = y_pos_bar_names,
          text =  'Null Model',
          textangle=angle,
            font = dict(size = size_bar_name ),
           ),

        ],    
)




fig = go.Figure(data=data, layout=layout)





fig['layout']['font']['size'] = font_gral-5   
fig['layout']['xaxis']['tickangle'] = 0
fig['layout']['xaxis']['tickfont']['size'] = font_gral -5
fig['layout']['yaxis']['tickfont']['size'] = font_gral -10
fig['layout']['margin']=dict(
        l=200,
       # r=50,
        b=100,
        t=200,
        pad=15
    )




#### Plot:

In [ ]:



fig_filename='null_model_fract_usage_top_bottom_ref_'+str(years[0])+string_code_categ.replace(" ","_")+"_"+str(Niter)+"iter"
offline.plot(fig, auto_open=True, image = 'png', image_filename='../plots/'+fig_filename ,image_width=2000, image_height=1200, filename=fig_filename+'.html', validate=True)



In [ ]:
This would be the result:

<img src="files/results_from_randomization_actual_usage_vs_expected.png">

FIGURE CAPTION (THIS RESULTS ARE UNPUBLISHED YET, PLEASE KEEP IT CONFIDENTIAL): The usage of references by top and bottom papers is significantly different than what could be expected at random. After controlling for publication year, field and group vs isolated references, we find that top papers used a higher-than-expected fraction of top references, and a lower-than-expected fraction of bottom references. The exact opposite is true for bottom papers: they use a lower-than-expected fraction of top references, and a higher-than-expected fraction of bottom references. Definition of top and bottom papers (or references) is given by the 10% top or bottom percentiles of number of citations accrued by said papers (or references).